<hr />

### NoteBookの見方
コード以外の情報 
<span >白/黒での記載は速習Symbol</span>  
<span style="color:red">赤色での記載は補足情報</span>  
<span >コード内で🌟マークがある場所は自分の情報に書き換えが必要</span>

<hr />

<span style="color:red">

# 環境構築
</span>
<span style="color:red">

## 1.Symbol SDKの読み込み </span>

In [1]:
(script = document.createElement("script")).src = "https://xembook.github.io/nem2-browserify/symbol-sdk-pack-2.0.3.js";
document.getElementsByTagName("head")[0].appendChild(script);

<span style="color:red">

## 2.Symbol用の共通設定 </span>

In [2]:
NODE = 'https://sym-test-03.opening-line.jp:3001';
sym = require("/node_modules/symbol-sdk");
repo = new sym.RepositoryFactoryHttp(NODE);
txRepo = repo.createTransactionRepository();
mosaicRepo = repo.createMosaicRepository();
accountRepo = repo.createAccountRepository();
(async () => {
  networkType = await repo.getNetworkType().toPromise();
  generationHash = await repo.getGenerationHash().toPromise();
  epochAdjustment = await repo.getEpochAdjustment().toPromise();
})();

function clog(signedTx){
    console.log(NODE + "/transactionStatus/" + signedTx.hash);
    console.log(NODE + "/transactions/confirmed/" + signedTx.hash);
    console.log("https://symbol.fyi/transactions/" + signedTx.hash);
    console.log("https://testnet.symbol.fyi/transactions/" + signedTx.hash);
}

<img width="800" alt="スクリーンショット 2023-01-21 1 39 39" src="https://user-images.githubusercontent.com/47712051/232594541-24c788ea-5981-4fee-9952-55d324f141fd.png">

<hr />

# 3.アカウント

アカウントは秘密鍵に紐づく情報が記録されたデータ構造体です。アカウントと関連づいた秘密鍵を使って署名することでのみブロックチェーンのデータを更新することができます。  

## 3.1 アカウント生成

アカウントには秘密鍵と公開鍵をセットにしたキーペア、アドレスなどの情報が含まれています。まずはランダムにアカウントを作成して、それらの情報を確認してみましょう。  

### 新規生成

In [3]:
alice = sym.Account.generateNewAccount(networkType);
console.log(JSON.stringify(alice,null,2));

{
  "address": {
    "address": "TAOP6YRVWUH6VMBAZASNLYHL2SEU7DLALXJN7EY",
    "networkType": 152
  },
  "keyPair": {
    "privateKey": {
      "0": 95,
      "1": 210,
      "2": 241,
      "3": 19,
      "4": 219,
      "5": 101,
      "6": 140,
      "7": 195,
      "8": 152,
      "9": 92,
      "10": 243,
      "11": 223,
      "12": 244,
      "13": 217,
      "14": 52,
      "15": 61,
      "16": 230,
      "17": 218,
      "18": 40,
      "19": 59,
      "20": 56,
      "21": 40,
      "22": 252,
      "23": 4,
      "24": 74,
      "25": 192,
      "26": 221,
      "27": 39,
      "28": 220,
      "29": 116,
      "30": 252,
      "31": 85
    },
    "publicKey": {
      "0": 112,
      "1": 176,
      "2": 198,
      "3": 189,
      "4": 178,
      "5": 194,
      "6": 145,
      "7": 241,
      "8": 243,
      "9": 13,
      "10": 157,
      "11": 166,
      "12": 207,
      "13": 125,
      "14": 28,
      "15": 109,
      "16": 191,
      "17": 84,
      "18": 88,
      "1

###### 出力例
```js
> Account
    address: Address {address: 'TBXUTAX6O6EUVPB6X7OBNX6UUXBMPPAFX7KE5TQ', networkType: 152}
    keyPair: {privateKey: Uint8Array(32), publicKey: Uint8Array(32)}
```

networkTypeは以下の通りです。
```js
{104: 'MAIN_NET', 152: 'TEST_NET'}
```

### 秘密鍵と公開鍵の導出

In [5]:
console.log(alice.privateKey);
console.log(alice.publicKey);

5FD2F113DB658CC3985CF3DFF4D9343DE6DA283B3828FC044AC0DD27DC74FC55

70B0C6BDB2C291F1F30D9DA6CF7D1C6DBF54588285B040ADD762EACD853EE4E9

```
> 1E9139CC1580B4AED6A1FE110085281D4982ED0D89CE07F3380EB83069B1****
> D4933FC1E4C56F9DF9314E9E0533173E1AB727BDB2A04B59F048124E93BEFBD2
```

#### 注意事項
秘密鍵を紛失するとそのアカウントに紐づけられたデータを操作することが出来なくなります。また、他人は知らないという秘密鍵の性質を利用してデータ操作の署名を行うので、秘密鍵を他人に教えてはいけません。組織のなかで秘密鍵を譲り受けて運用を続けるといった行為も控えましょう。
一般的なWebサービスでは「アカウントID」に対してパスワードが割り振られるため、パスワードの変更が可能ですが、ブロックチェーンではパスワードにあたる秘密鍵に対して一意に決まるID(アドレス)が割り振られるため、アカウントに紐づく秘密鍵を変更するということはできません。  


### アドレスの導出

In [4]:
aliceRawAddress = alice.address.plain();
console.log(aliceRawAddress);

TAOP6YRVWUH6VMBAZASNLYHL2SEU7DLALXJN7EY

```js
> TBXUTAX6O6EUVPB6X7OBNX6UUXBMPPAFX7KE5TQ
```

これらがブロックチェーンを操作するための最も基本的な情報となります。また、秘密鍵からアカウントを生成したり、公開鍵やアドレスのみを扱うクラスの生成方法も確認しておきましょう。  

### 秘密鍵からアカウント生成

In [ ]:
alice = sym.Account.createFromPrivateKey(
    "1E9139CC1580B4AED6A1FE110085281D4982ED0D89CE07F3380EB83069B1****", //🌟ここを先ほど導出したalice.privateKeyに置き換えて実行する
    networkType
);

### 公開鍵クラスの生成

In [ ]:
alicePublicAccount = sym.PublicAccount.createFromPublicKey(
    "D4933FC1E4C56F9DF9314E9E0533173E1AB727BDB2A04B59F048124E9****", //🌟ここを先ほど導出したalice.publicKeyに置き換えて実行する
    networkType
);
console.log(JSON.stringify(alicePublicAccount,null,2));

###### 出力例
```js
> PublicAccount
    address: Address {address: 'TBXUTAX6O6EUVPB6X7OBNX6UUXBMPPAFX7KE5TQ', networkType: 152}
    publicKey: "D4933FC1E4C56F9DF9314E9E0533173E1AB727BDB2A04B59F048124E93BEFBD2"
```

### アドレスクラスの生成

In [ ]:
aliceAddress = sym.Address.createFromRawAddress(
    "TBXUTAX6O6EUVPB6X7OBNX6UUXBMPPAFX7KE*******" //🌟ここを先ほど導出したaliceRawAddressに置き換えて実行する
);
console.log(JSON.stringify(aliceAddress,null,2));

###### 出力例
```js
> Address
    address: "TBXUTAX6O6EUVPB6X7OBNX6UUXBMPPAFX7KE5TQ"
    networkType: 152
```

## 3.2 アカウントへの送信

アカウントを作成しただけでは、ブロックチェーンにデータを送信することはできません。  
パブリックブロックチェーンはリソースを有効活用するためにデータ送信時に手数料を要求します。  
Symbolブロックチェーンでは、この手数料をXYMという共通トークンで支払うことになります。  
アカウントを生成したら、この後の章から説明するトランザクションを実行するために必要な手数料を送信しておきます。  

### フォーセットから送信

テストネットではフォーセット（蛇口）サービスから検証用のXYMを入手することができます。  
メインネットの場合は取引所などでXYMを購入するか、投げ銭サービス(NEMLOG,QUEST)などを利用して寄付を募りましょう。  

テストネット
- FAUCET(蛇口)
  - https://testnet.symbol.tools/

メインネット
- NEMLOG
  - https://nemlog.nem.social/
- QUEST
  - https://quest-bc.com/



### エクスプローラーで確認

フォーセットから作成したアカウントへ送信が成功したらエクスプローラーで確認してみましょう。

- テストネット
  - https://testnet.symbol.fyi/
- メインネット
  - https://symbol.fyi/

## 3.3 アカウント情報の確認

ノードに保存されているアカウント情報を取得します。

### 所有モザイク一覧の取得

In [ ]:
accountRepo = repo.createAccountRepository();
accountInfo = await accountRepo.getAccountInfo(aliceAddress).toPromise();
console.log(JSON.stringify(accountInfo,null,2));

#### publicKey
クライアント側で作成しただけで、ブロックチェーンでまだ利用されていないアカウント情報は記録されていません。宛先として指定されて受信することで初めてアカウント情報が記録され、署名したトランザクションを送信することで公開鍵の情報が記録されます。そのため、publicKeyは現在`00000...`表記となっています。

#### UInt64
JavaScriptでは大きすぎる数値はあふれてしまうため、idやamountはUInt64というsdkの独自フォーマットで管理されています。文字列に変換する場合は toString()、数値に変換する場合は compact()、16進数にする場合は toHex() で変換してください。

In [ ]:
console.log("addressHeight:"); //アドレスが記録されたブロック高
console.log(accountInfo.addressHeight.compact()); //数値
accountInfo.mosaics.forEach(mosaic => {
  console.log("id:" + mosaic.id.toHex()); //16進数
  console.log("amount:" + mosaic.amount.toString()); //文字列
});

大きすぎるid値をcompactで数値変換するとエラーが発生することがあります。  
`Compacted value is greater than Number.Max_Value.`


#### 表示桁数の調整

所有するトークンの量は誤差の発生を防ぐため、整数値で扱います。トークンの定義から可分性を取得することができるので、その値を使って正確な所有量を表示してみます。  

In [ ]:
mosaicRepo = repo.createMosaicRepository();
mosaicAmount = accountInfo.mosaics[0].amount.toString();
mosaicInfo = await mosaicRepo.getMosaic(accountInfo.mosaics[0].id).toPromise();
divisibility = mosaicInfo.divisibility; //可分性
if(divisibility > 0){
  displayAmount = mosaicAmount.slice(0,mosaicAmount.length-divisibility)  
  + "." + mosaicAmount.slice(-divisibility);
}else{
  displayAmount = mosaicAmount;
}
console.log(displayAmount);

## 3.4 現場で使えるヒント
### 暗号化と署名

アカウントとして生成した秘密鍵や公開鍵は、そのまま従来の暗号化や電子署名として活用することができます。信頼性に問題点があるアプリケーションを使用する必要がある場合も、個人間（エンドツーエンド）でデータの秘匿性・正当性を検証することができます。  

<img width="800" alt="スクリーンショット 2023-01-21 1 39 39" src="https://user-images.githubusercontent.com/47712051/232594570-a53e4329-b653-481b-83d3-bd8a9ce611db.png"> 

<img width="800" alt="スクリーンショット 2023-01-21 1 39 39" src="https://user-images.githubusercontent.com/47712051/232594585-1552a006-ffdf-4bc2-a074-098f3dc07b00.png">

<img width="800" alt="スクリーンショット 2023-01-21 1 39 39" src="https://user-images.githubusercontent.com/47712051/232594678-38059798-a08d-471e-846a-0eaa35628e5b.png">  

#### 事前準備：対話のためのBobアカウントを生成

In [ ]:
bob = sym.Account.generateNewAccount(networkType);
bobPublicAccount = bob.publicAccount;

#### 暗号化

Aliceの秘密鍵・Bobの公開鍵で暗号化し、Aliceの公開鍵・Bobの秘密鍵で復号します（AES-GCM形式）。

In [ ]:
message = 'Hello Symol!';
encryptedMessage = alice.encryptMessage(message ,bob.publicAccount);
console.log(encryptedMessage);

```js
> 294C8979156C0D941270BAC191F7C689E93371EDBC36ADD8B920CF494012A97BA2D1A3759F9A6D55D5957E9D
```

#### 復号化

In [ ]:
decryptMessage = bob.decryptMessage(
    new sym.EncryptedMessage(
      "294C8979156C0D941270BAC191F7C689E93371EDBC36ADD8B920CF494012A97BA2D1A3759F9A6D55D5957E9D" //🌟ここを先ほど導出したencryptedMessageのpayloadに置き換えて実行する
    ),
    alice.publicAccount
  ).payload
  console.log(decryptMessage);  

```js
> "Hello Symol!"
```

<span style="color:red">

### 補足　別のアカウントでは復号化可能か検証</span>  

In [ ]:
carol = sym.Account.generateNewAccount(networkType);

decryptMessage = carol.decryptMessage(
    new sym.EncryptedMessage(
      "294C8979156C0D941270BAC191F7C689E93371EDBC36ADD8B920CF494012A97BA2D1A3759F9A6D55D5957E9D" //🌟ここを先ほど導出したencryptedMessageのpayloadに置き換えて実行する
    ),
    alice.publicAccount
  ).payload
  console.log(decryptMessage);

#### 署名

Aliceの秘密鍵でメッセージを署名し、Aliceの公開鍵と署名でメッセージを検証します。

In [ ]:
Buffer = require("/node_modules/buffer").Buffer;
payload = Buffer.from("Hello Symol!", 'utf-8');
signature = Buffer.from(sym.KeyPair.sign(alice.keyPair, payload)).toString("hex").toUpperCase();
console.log(signature);

```
> B8A9BCDE9246BB5780A8DED0F4D5DFC80020BBB7360B863EC1F9C62CAFA8686049F39A9F403CB4E66104754A6AEDEF8F6B4AC79E9416DEEDC176FDD24AFEC60E
```

#### 検証

In [ ]:
isVerified = sym.KeyPair.verify(
    alice.keyPair.publicKey,
    Buffer.from("Hello Symol!", 'utf-8'),
    Buffer.from(signature, 'hex')
)
console.log(isVerified);  

```js
> true
```

ブロックチェーンを使用しない署名は何度も再利用される可能性があることにご注意ください。

<span style="color:red">

### 補足　KeyPairが異なった場合の挙動を検証</span>  

In [ ]:
isVerified = sym.KeyPair.verify(
    bob.keyPair.publicKey,
    Buffer.from("Hello Symol!", 'utf-8'),
    Buffer.from(signature, 'hex')
)
console.log(isVerified);  

### アカウントの保管

アカウントの管理方法について説明しておきます。  
秘密鍵はそのままで保存しないようにしてください。symbol-qr-libraryを利用して秘密鍵をパスフレーズで暗号化して保存する方法を紹介します。  

#### 秘密鍵の暗号化

In [ ]:
qr = require("/node_modules/symbol-qr-library");
//パスフレーズでロックされたアカウント生成
signerQR = qr.QRCodeGenerator.createExportAccount(
  alice.privateKey, networkType, generationHash, "パスフレーズ" //🌟このパスフレーズを自分のパスフレーズに置き換えて実行する
);
//QRコード表示
signerQR.toBase64().subscribe(x =>{
  //HTML body上にQRコードを表示する例
  (tag= document.createElement('img')).src = x;
  document.getElementsByTagName('body')[0].appendChild(tag);
});
//アカウントを暗号化したJSONデータとして表示
jsonSignerQR = signerQR.toJSON();
console.log(jsonSignerQR);

###### 出力例
```js
> {"v":3,"type":2,"network_id":152,"chain_id":"7FCCD304802016BEBBCD342A332F91FF1F3BB5E902988B352697BE245F48E836","data":{"ciphertext":"e9e2f76cb482fd054bc13b7ca7c9d086E7VxeGS/N8n1WGTc5MwshNMxUiOpSV2CNagtc6dDZ7rVZcnHXrrESS06CtDTLdD7qrNZEZAi166ucDUgk4Yst0P/XJfesCpXRxlzzNgcK8Q=","salt":"54de9318a44cc8990e01baba1bcb92fa111d5bcc0b02ffc6544d2816989dc0e9"}}
```
このjsonSignerQRで出力されるQRコード、あるいはテキストを保存しておけばいつでも秘密鍵を復元することができます。

#### 暗号化された秘密鍵の復号

In [ ]:
//🌟保存しておいたテキスト、あるいはQRコードスキャンで得られたテキストをjsonSignerQRに代入
jsonSignerQR = '{"v":3,"type":2,"network_id":152,"chain_id":"7FCCD304802016BEBBCD342A332F91FF1F3BB5E902988B352697BE245F48E836","data":{"ciphertext":"e9e2f76cb482fd054bc13b7ca7c9d086E7VxeGS/N8n1WGTc5MwshNMxUiOpSV2CNagtc6dDZ7rVZcnHXrrESS06CtDTLdD7qrNZEZAi166ucDUgk4Yst0P/XJfesCpXRxlzzNgcK8Q=","salt":"54de9318a44cc8990e01baba1bcb92fa111d5bcc0b02ffc6544d2816989dc0e9"}}';
qr = require("/node_modules/symbol-qr-library");
signerQR = qr.AccountQR.fromJSON(jsonSignerQR,"パスフレーズ"); //🌟このパスフレーズを自分のパスフレーズに置き換えて実行する
console.log(signerQR.accountPrivateKey);

###### 出力例
```js
> 1E9139CC1580B4AED6A1FE110085281D4982ED0D89CE07F3380EB83069B1****
```

<span style="color:red">

#### 以降の章ではこの章で作成した秘密鍵を使ってaliceアカウントをリストアして演習を行なっていきますので、以下の作成した秘密鍵合宿中無くさないようにメモ帳などに保存しておいて下さい。</span> 

In [ ]:
console.log(alice.privateKey);